In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import os
import TrialPlots as plots
import scipy.io
from scipy import signal

In [3]:
def define_defaults():
    defaults = dict()
    defaults['fs'] = 1e3  # sampling frequency
    defaults['fc'] = 5  # low pass cut-off (Hz)
    # feedforward estimate of hand position used by KINARM (constant)
    defaults['fdfwd'] = 0.06
    # Define condition order for plotting
    defaults['reachorder'] = ['Reaching', 'Interception']
    defaults['grouporder'] = ['CP', 'TDC']
    defaults['armorder'] = ['More Affected', 'Less Affected']
    return defaults

In [4]:
def lowPassFilter(data, fc, fs, filter_order=4):
    """ fc is cut-off frequency of filter
    fs is sampling rate
    """
    w = fc/(fs/2)  # Normalize the frequency
    # divide filter order by 2
    [b, a] = signal.butter(filter_order/2, w, 'low')
    dataLPfiltfilt = signal.filtfilt(b, a, data)  # apply filtfilt to data
    return dataLPfiltfilt


def dist(x1, y1, x2, y2):
    dist = np.sqrt(np.square(x1-x2)+np.square(y1-y2))
    return dist

In [5]:
def getDataCP(mdf, matfiles, defaults):
    all_df = pd.DataFrame()
   
    row_name_str=  ["cpvib040"]
    for index, row in mdf.iterrows():
        if row['KINARM ID'] not in row_name_str:
            continue
        elif row['KINARM ID'].startswith('CHEAT'):
            subject = row['KINARM ID'][-3:]
        else:
            subject = row['KINARM ID']
        print(f'evaluating the subject : {subject}')
        subjectmat = 'CHEAT-CP'+subject+row['Visit_Day']+'.mat'

        mat = os.path.join(matfiles, subjectmat)

        if not os.path.exists(mat):
            print('skipping', mat)
            # if row['Visit_Day'] =='Day1':
            #    assert 0
            continue

        loadmat = scipy.io.loadmat(mat)

        data = loadmat['subjDataMatrix'][0][0]
        # print(data.keys())
        # assert 0
        # data[23] is all data for trial 24
        # data[23].T[4] is all data for the 5th column of that trial

        # simple, for each trial, collect (a) condition--12, (b) TP--11 (c) hitormiss-->14 (d) feedback time from 15
        # (e) hitormis and feedback time from 10 SKIP (f) trial duration 13 ; affectedhand 16 (1 if using affected)

        allTrials = []
        subjectTrajs = []
        for i in range(len(data)):
            thisData = data[i]
            trajData = getHandTrajectories(thisData, defaults)
            # print(f"Calling the Kinematics function for the sub : {subjectmat}")
            kinData = getHandKinematics(thisData, defaults,i,subject)
            row_values = [thisData.T[12][0], thisData.T[16][0], thisData.T[11][0],
                          thisData.T[13][0], thisData.T[14][0], thisData.T[15][0],
                          kinData['RT'], kinData['CT'], kinData['velPeak'],
                          kinData['xPosError'], kinData['minDist'], kinData['targetDist'], kinData['handDist'], kinData['straightlength'],
                          kinData['pathlength'], kinData['targetlength'], kinData['CursorX'], kinData['CursorY'],
                          kinData['IA_RT'], kinData['IA_50RT'], kinData['RTalt'], kinData['IA_RTalt'],
                          kinData['maxpathoffset'], kinData['meanpathoffset'], kinData['xTargetEnd'], kinData['yTargetEnd'], kinData['EndPointError'], kinData['IDE'], kinData['PLR'], kinData['isCurveAround'],i]

            allTrials.append(row_values)
            subjectTrajs.append(trajData)
        print(f'evaluated the kindData for sub : {subject}')

        df = pd.DataFrame(allTrials, columns=['Condition', 'Affected', 'TP', 'Duration', 'Accuracy', 'FeedbackTime',
                                              'RT', 'CT', 'velPeak', 'xPosError', 'minDist', 'targetDist', 'handDist', 'straightlength',
                                              'pathlength', 'targetlength', 'cursorX', 'cursorY', 'IA_RT', 'IA_50RT',
                                              'RTalt', 'IA_RTalt', 'maxpathoffset', 'meanpathoffset', 'xTargetEnd', 'yTargetEnd', 'EndPointError', 'IDE', 'PLR', 'isCurveAround','trial_number'])
        # data cleaning
        df['Affected'] = df['Affected'].map(
            {1: 'More Affected', 0: 'Less Affected'})
        df['Condition'] = df['Condition'].map(
            {1: 'Reaching', 2: 'Interception'})
        df['Duration'] = df['TP'].map(
            {1: 625, 2: 500, 3: 750, 4: 900, 5: 625, 6: 500, 7: 750, 8: 900})
        df['MT'] = df['CT'] - df['RT']
        df['subject'] = subject
        df['age'] = row['Age at Visit (yr)']
        df['visit'] = row['Visit ID']
        df['day'] = row['Visit_Day']
        df['studyid'] = row['Subject ID']

        if row['Group'] == 0:
            df['group'] = 'TDC'
        else:
            df['group'] = 'CP'

        df['pathratio'] = df['pathlength'] / df['targetlength']
        all_df = pd.concat([all_df, df])

        # combine all trajectories
        allTrajs[subject+row['Visit_Day']] = subjectTrajs

    return all_df, allTrajs
